In [1]:
import sys
sys.path.append("..")
from pathlib import Path

import numpy as np
import polars as pl

## 学習データ

In [2]:
data_dir = Path("../../data/lb")
trn_files = sorted(list(data_dir.glob("processed_train_*.parquet")))
len(trn_files)

10

In [3]:
for trn_filename in trn_files:
    trn_df = pl.read_parquet(trn_filename)
    break

In [4]:
trn_df.shape

(30000000, 7)

In [5]:
trn_df.head()

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
i32,str,str,str,str,str,bool
0,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.Br.NCC1CCCN…","""C#CCOc1ccc(CNc…","""BRD4""",false
1,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.Br.NCC1CCCN…","""C#CCOc1ccc(CNc…","""HSA""",false
2,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.Br.NCC1CCCN…","""C#CCOc1ccc(CNc…","""sEH""",false
3,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.NCc1cccc(Br…","""C#CCOc1ccc(CNc…","""BRD4""",false
4,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.NCc1cccc(Br…","""C#CCOc1ccc(CNc…","""HSA""",false


In [6]:
trn_df[0].to_numpy()[0].tolist()

[0,
 'C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21',
 'C#CCOc1ccc(CN)cc1.Cl',
 'Br.Br.NCC1CCCN1c1cccnn1',
 'C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1',
 'BRD4',
 False]

In [7]:
df = None
for i, protein_name in enumerate(["BRD4", "HSA", "sEH"]):
    sub_df = trn_df[i::3]
    sub_df = sub_df.rename({"binds": protein_name})
    if i == 0:
        df = sub_df.select(pl.exclude("protein_name"))
    else:
        df = pl.concat([df, sub_df[[protein_name]]], how="horizontal")
df

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,BRD4,HSA,sEH
i32,str,str,str,str,bool,bool,bool
0,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.Br.NCC1CCCN…","""C#CCOc1ccc(CNc…",false,false,false
3,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.NCc1cccc(Br…","""C#CCOc1ccc(CNc…",false,false,false
6,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C#CCOc1ccc(CN)…","""C#CCOc1ccc(CNc…",false,false,false
9,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C=C(C)C(=O)NCC…","""C#CCOc1ccc(CNc…",false,false,false
12,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C=CCNC(=O)CN.C…","""C#CCOc1ccc(CNc…",false,false,false
…,…,…,…,…,…,…,…
29999985,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc(-c2c…","""COc1cc(C(=O)N[…",false,false,false
29999988,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc2cc(F…","""COc1cc(C(=O)N[…",false,false,false
29999991,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc2cccc…","""COc1cc(C(=O)N[…",false,false,false


In [8]:
df

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,BRD4,HSA,sEH
i32,str,str,str,str,bool,bool,bool
0,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.Br.NCC1CCCN…","""C#CCOc1ccc(CNc…",false,false,false
3,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""Br.NCc1cccc(Br…","""C#CCOc1ccc(CNc…",false,false,false
6,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C#CCOc1ccc(CN)…","""C#CCOc1ccc(CNc…",false,false,false
9,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C=C(C)C(=O)NCC…","""C#CCOc1ccc(CNc…",false,false,false
12,"""C#CC[C@@H](CC(…","""C#CCOc1ccc(CN)…","""C=CCNC(=O)CN.C…","""C#CCOc1ccc(CNc…",false,false,false
…,…,…,…,…,…,…,…
29999985,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc(-c2c…","""COc1cc(C(=O)N[…",false,false,false
29999988,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc2cc(F…","""COc1cc(C(=O)N[…",false,false,false
29999991,"""COc1cc(C(=O)O)…","""Cl.NCCC1CC1""","""Cl.NCc1nc2cccc…","""COc1cc(C(=O)N[…",false,false,false


In [9]:
for i in range(0, 100, 3):
    x = trn_df["molecule_smiles", i: i+3].to_list()
    assert x[0] == x[1] and x[1] == x[2]

In [10]:
trn_df["binds"].value_counts().sort("binds").with_columns(pl.col("count") / len(trn_df) * 100)

binds,count
bool,f64
false,99.42249
true,0.57751


In [11]:
trn_df["buildingblock1_smiles"].value_counts().sort("count")

buildingblock1_smiles,count
str,u32
"""COc1cc(C(=O)O)…",587655
"""C=CC[C@@H](NC(…",1089030
"""C#CC[C@@H](NC(…",1089060
"""CC(C)(C)OC(=O)…",1089069
"""C=CCC[C@@H](NC…",1089093
…,…
"""COc1c(F)ccc(C(…",1089729
"""CCS(=O)(=O)c1c…",1089732
"""CC(C)(C)OC(=O)…",1089762


In [12]:
trn_df["buildingblock2_smiles"].value_counts().sort("count")

buildingblock2_smiles,count
str,u32
"""Cl.Nc1ccc2cccn…",14586
"""Cl.NCc1ccc[nH]…",14643
"""Nc1cncc(F)c1""",14661
"""Nc1ccc2nsnc2c1…",14964
"""Cl.NCc1ccsc1C(…",15012
…,…
"""Cc1cc(N)on1""",71937
"""Cl.NCC1Cc2cccc…",72189
"""Cl.Cl.NCC1CCNC…",72483


In [13]:
trn_df["buildingblock3_smiles"].value_counts().sort("count")

buildingblock3_smiles,count
str,u32
"""NCc1ccccn1""",81
"""CC1(CCCCN)OCCO…",165
"""Cl.NCc1cnc2n1C…",192
"""Nc1nc(-c2ccc(F…",309
"""NCc1c(F)cccc1N…",408
…,…
"""COc1nc(C)ccc1C…",57165
"""Cc1cc(CN)c(C)[…",57165
"""COc1cc(CN)sn1.…",57168


In [14]:
trn_df["molecule_smiles"].value_counts().sort("molecule_smiles")

molecule_smiles,count
str,u32
"""C#CCOc1ccc(CNc…",3
"""C#CCOc1ccc(CNc…",3
"""C#CCOc1ccc(CNc…",3
"""C#CCOc1ccc(CNc…",3
"""C#CCOc1ccc(CNc…",3
…,…
"""Cn1nnc(Nc2nc(N…",3
"""Cn1nnc(Nc2nc(N…",3
"""Cn1nnc(Nc2nc(N…",3


In [15]:
trn_df["protein_name"].value_counts().sort("protein_name")

protein_name,count
str,u32
"""BRD4""",10000000
"""HSA""",10000000
"""sEH""",10000000


## テストデータ

In [16]:
test_df = pl.read_parquet("../../data/lb/test.parquet")
test_df.shape

(1674896, 6)

In [17]:
test_df.head()

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
i64,str,str,str,str,str
295246830,"""C#CCCC[C@H](NC…","""C=Cc1ccc(N)cc1…","""C=Cc1ccc(N)cc1…","""C#CCCC[C@H](Nc…","""BRD4"""
295246831,"""C#CCCC[C@H](NC…","""C=Cc1ccc(N)cc1…","""C=Cc1ccc(N)cc1…","""C#CCCC[C@H](Nc…","""HSA"""
295246832,"""C#CCCC[C@H](NC…","""C=Cc1ccc(N)cc1…","""C=Cc1ccc(N)cc1…","""C#CCCC[C@H](Nc…","""sEH"""
295246833,"""C#CCCC[C@H](NC…","""C=Cc1ccc(N)cc1…","""CC(O)Cn1cnc2c(…","""C#CCCC[C@H](Nc…","""BRD4"""
295246834,"""C#CCCC[C@H](NC…","""C=Cc1ccc(N)cc1…","""CC(O)Cn1cnc2c(…","""C#CCCC[C@H](Nc…","""HSA"""


In [18]:
test_df["buildingblock1_smiles"].value_counts().sort("count")

buildingblock1_smiles,count
str,u32
"""COc1cc(C(=O)O)…",1989
"""O=C(O)C[C@@H](…",1989
"""O=C(Nc1ccc(C(=…",1989
"""O=C(Nc1nccnc1C…",1989
"""O=C(O)C[C@H](C…",1989
…,…
"""O=C(O)[C@@H]1C…",14004
"""O=C(O)[C@@H]1C…",14039
"""CC(C)(C)OC(=O)…",14041


In [19]:
test_df["buildingblock2_smiles"].value_counts().sort("count")

buildingblock2_smiles,count
str,u32
"""CN1CCN(c2cccc(…",51
"""Cl.NC[C@@H]1C[…",102
"""Cc1ccc(N)cn1""",153
"""CC1(C)CCCC1(O)…",153
"""COC(=O)c1cc(Cl…",204
…,…
"""Nc1ccc2[nH]c(=…",7302
"""Nc1nc(-c2cccnc…",8031
"""CN1C(=O)Cc2cc(…",10077


In [20]:
test_df["buildingblock3_smiles"].value_counts().sort("count")

buildingblock3_smiles,count
str,u32
"""Nc1cccc2cnccc1…",3
"""NCc1c(F)cccc1N…",3
"""Cl.N#CC1(NC(=O…",6
"""Cl.NCC1Cc2cccc…",9
"""Cl.Cl.NCc1cnsc…",15
…,…
"""Nc1nc(NCc2ccc(…",6426
"""Nc1ccc2[nH]c(=…",6714
"""NCC1CN2CCN1CC2…",8361


In [21]:
test_df["molecule_smiles"].value_counts().sort("molecule_smiles")

molecule_smiles,count
str,u32
"""C#CCCC[C@H](Nc…",3
"""C#CCCC[C@H](Nc…",3
"""C#CCCC[C@H](Nc…",3
"""C#CCCC[C@H](Nc…",3
"""C#CCCC[C@H](Nc…",3
…,…
"""[N-]=[N+]=NCCC…",3
"""[N-]=[N+]=NCCC…",3
"""[N-]=[N+]=NCCC…",3
